# Traffic assignment SO congestion + CO2 (min)

$$
P_k =
\frac{\exp(-\mu \beta_{\text{time}} C_k)}
{\sum_{j \in \mathcal{K}_{od}} \exp(-\mu \beta_{\text{time}} C_j)}
$$


k_paths=5  
mu=100.0  
beta_time=1.0  
max_iter=500  
tol=1e-6  

In [1]:
import math
import numpy as np
import pandas as pd
import networkx as nx

# ============================
# 0) CO2 parameters (Paper: Barth & Boriboonsomsin, Real-World)  [Eq(1) + Table 1]
# ============================
# Paper (Real-World):
#   ln(y) = b0 + b1*x + b2*x^2 + b3*x^3 + b4*x^4
#   y: CO2 [g/mi], x: speed [mph]
#
# We convert to:
#   ln(EF) = A0 + A1*v + A2*v^2 + A3*v^3 + A4*v^4
#   EF: CO2 [g/km], v: speed [km/h]
#
# Conversion:
#   x = v / k,  y = k * EF,  k = 1.609344 (km per mile)
#   ln(EF) = ln(y) - ln(k)
# => A0 = b0 - ln(k)
#    A1 = b1 / k
#    A2 = b2 / k^2
#    A3 = b3 / k^3
#    A4 = b4 / k^4
#
# Real-World coefficients from Table 1 (paper)  :contentReference[oaicite:1]{index=1}
B0 = 7.613534994965560
B1 = -0.138565467462594
B2 = 0.003915102063854
B3 = -0.000049451361017
B4 = 0.000000238630156

K_KM_PER_MILE = 1.609344

def convert_b_to_A(b0, b1, b2, b3, b4, k=K_KM_PER_MILE):
    A0 = b0 - math.log(k)
    A1 = b1 / k
    A2 = b2 / (k**2)
    A3 = b3 / (k**3)
    A4 = b4 / (k**4)
    return A0, A1, A2, A3, A4

A0, A1, A2, A3, A4 = convert_b_to_A(B0, B1, B2, B3, B4)

p_CO2 = 15.0          # yen / kg-CO2
VOT = 43.74           # yen / min
lambda_CO2 = p_CO2 / VOT   # min / kg-CO2

V_MIN = 5.0           # km/h (safety)
V_MAX = 130.0         # km/h


# ----------------------------
# 1) Read & normalize inputs
# ----------------------------
def read_net_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()

    if ";" in df.columns:
        df = df.drop(columns=[";"])

    required = ["A", "B", "capacity", "t0_min", "bpr_B", "bpr_power", "length_km"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"net.csv missing columns: {missing}. got: {df.columns.tolist()}")

    out = pd.DataFrame()
    out["u"] = df["A"].astype(int)
    out["v"] = df["B"].astype(int)
    out["cap"] = df["capacity"].astype(float)
    out["t0"] = df["t0_min"].astype(float)
    out["alpha"] = df["bpr_B"].astype(float)
    out["beta"] = df["bpr_power"].astype(float)
    out["length"] = df["length_km"].astype(float)

    return out


def read_od_csv(path: str) -> pd.DataFrame:
    od = pd.read_csv(path)
    od.columns = od.columns.str.strip()

    col_map = {}
    for c in od.columns:
        lc = c.lower()
        if lc in ["o", "origin"]:
            col_map[c] = "origin"
        elif lc in ["d", "dest", "destination"]:
            col_map[c] = "destination"
        elif lc in ["ton", "demand", "trips", "q"]:
            col_map[c] = "demand"
    od = od.rename(columns=col_map)

    required = ["origin", "destination", "demand"]
    missing = [c for c in required if c not in od.columns]
    if missing:
        raise ValueError(f"od.csv missing columns: {missing}. got: {od.columns.tolist()}")

    od = od[required].copy()
    od["origin"] = od["origin"].astype(int)
    od["destination"] = od["destination"].astype(int)
    od["demand"] = od["demand"].astype(float)

    od = od[(od["demand"] > 0) & (od["origin"] != od["destination"])].reset_index(drop=True)
    return od


# ----------------------------
# 2) BPR & marginal congestion
# ----------------------------
def bpr_time(t0, x, cap, alpha=0.15, beta=4.0):
    if cap <= 0:
        return float("inf")
    return t0 * (1.0 + alpha * (x / cap) ** beta)

def bpr_marginal_time(t0, x, cap, alpha=0.15, beta=4.0):
    if cap <= 0:
        return float("inf")
    return t0 * (1.0 + alpha * (1.0 + beta) * (x / cap) ** beta)


# ----------------------------
# 3) CO2-related functions (UPDATED: paper-based EF in g/km, v in km/h)
# ----------------------------
def clip_speed(v_kmh: float) -> float:
    return max(V_MIN, min(V_MAX, float(v_kmh)))

def emission_factor(v_kmh: float) -> float:
    """
    EF(v) [g/km]
    EF(v) = exp(A0 + A1*v + A2*v^2 + A3*v^3 + A4*v^4)
    """
    v = max(0.1, clip_speed(v_kmh))  # avoid v=0 for numerical stability
    ln_ef = A0 + A1*v + A2*(v**2) + A3*(v**3) + A4*(v**4)
    return float(math.exp(ln_ef))

def co2_cost_min(length_km: float, t_min: float) -> float:
    """
    CO2 cost converted into minutes (min/veh)
    """
    if t_min <= 0 or length_km <= 0:
        return 0.0

    v_kmh = clip_speed(60.0 * length_km / t_min)  # km/h
    ef = emission_factor(v_kmh)                   # g/km
    co2_kg = ef * length_km / 1000.0              # kg/veh
    return lambda_CO2 * co2_kg                    # min/veh


# ----------------------------
# 4) Logit-SUE via MSA (SO + CO2)
# ----------------------------
def run_logit_sue_so_congestion_co2(net: pd.DataFrame,
                                    od: pd.DataFrame,
                                    k_paths=10,
                                    mu=1.0,
                                    beta_time=1.0,
                                    max_iter=60,
                                    tol=1e-5):

    edges = list(zip(net["u"].tolist(), net["v"].tolist()))
    x = {e: 0.0 for e in edges}

    G = nx.DiGraph()
    for (u, v), t0 in zip(edges, net["t0"].tolist()):
        G.add_edge(u, v, weight=float(t0))

    def compute_edge_costs(x_dict):
        c = {}
        for i, (u, v) in enumerate(edges):
            row = net.iloc[i]

            # congestion SO marginal cost
            mc = bpr_marginal_time(
                row["t0"], x_dict[(u, v)],
                row["cap"], row["alpha"], row["beta"]
            )

            # CO2 cost (min) using updated EF(v)
            co2 = co2_cost_min(row["length"], mc)

            c[(u, v)] = mc + co2

        return c

    def rel_change(x_old, x_new):
        num = sum(abs(x_new[e] - x_old[e]) for e in edges)
        den = max(1.0, sum(abs(x_old[e]) for e in edges))
        return num / den

    for it in range(1, max_iter + 1):
        edge_costs = compute_edge_costs(x)

        for (u, v), cost in edge_costs.items():
            G[u][v]["weight"] = float(cost)

        x_hat = {e: 0.0 for e in edges}

        for _, r in od.iterrows():
            o, d, q = int(r["origin"]), int(r["destination"]), float(r["demand"])
            if q <= 0:
                continue

            cand_paths = nx.shortest_simple_paths(G, o, d, weight="weight")
            paths = []
            for _ in range(k_paths):
                try:
                    p = next(cand_paths)
                    paths.append(list(zip(p[:-1], p[1:])))
                except StopIteration:
                    break

            if not paths:
                continue

            utils = []
            for p in paths:
                cost_p = sum(edge_costs[e] for e in p)
                utils.append(-beta_time * cost_p)

            umax = max(utils)
            probs = np.exp(mu * (np.array(utils) - umax))
            probs = probs / probs.sum()

            for p, pk in zip(paths, probs):
                for e in p:
                    x_hat[e] += q * pk

        step = 1.0 / it
        x_new = {e: x[e] + step * (x_hat[e] - x[e]) for e in edges}

        gap = rel_change(x, x_new)
        x = x_new
        print(f"[SO+CO2] Iter {it:02d} | step={step:.4f} | rel_change={gap:.6e}")

        if gap < tol:
            break

    out = net.copy()
    out["flow"] = [x[(u, v)] for (u, v) in edges]

    out["time"] = [
        bpr_time(out.loc[i, "t0"], out.loc[i, "flow"],
                 out.loc[i, "cap"], out.loc[i, "alpha"], out.loc[i, "beta"])
        for i in range(len(out))
    ]

    out["mc_time"] = [
        bpr_marginal_time(out.loc[i, "t0"], out.loc[i, "flow"],
                          out.loc[i, "cap"], out.loc[i, "alpha"], out.loc[i, "beta"])
        for i in range(len(out))
    ]

    return out


# ----------------------------
# 5) main
# ----------------------------
if __name__ == "__main__":
    net = read_net_csv("/Users/keito-shiraki/traffic_assignment/data/csv/SiouxFalls_net_Vff60_with_minutes.csv")
    od = read_od_csv("/Users/keito-shiraki/traffic_assignment/data/csv/SiouxFalls_od.csv")

    result = run_logit_sue_so_congestion_co2(
        net=net,
        od=od,
        k_paths=5,
        mu=100.0,
        beta_time=1.0,
        max_iter=500,
        tol=1e-6
    )

    result.to_csv("assignment_result_SO_congestion_CO2_min2.csv", index=False)
    print("Saved: assignment_result_SO_congestion_CO2_min2.csv")


[SO+CO2] Iter 01 | step=1.0000 | rel_change=8.881000e+05
[SO+CO2] Iter 02 | step=0.5000 | rel_change=8.377849e-01
[SO+CO2] Iter 03 | step=0.3333 | rel_change=4.164270e-01
[SO+CO2] Iter 04 | step=0.2500 | rel_change=2.184624e-01
[SO+CO2] Iter 05 | step=0.2000 | rel_change=1.486770e-01
[SO+CO2] Iter 06 | step=0.1667 | rel_change=9.775184e-02
[SO+CO2] Iter 07 | step=0.1429 | rel_change=6.300168e-02
[SO+CO2] Iter 08 | step=0.1250 | rel_change=4.553057e-02
[SO+CO2] Iter 09 | step=0.1111 | rel_change=3.817381e-02
[SO+CO2] Iter 10 | step=0.1000 | rel_change=2.580327e-02
[SO+CO2] Iter 11 | step=0.0909 | rel_change=2.479369e-02
[SO+CO2] Iter 12 | step=0.0833 | rel_change=2.358095e-02
[SO+CO2] Iter 13 | step=0.0769 | rel_change=2.581362e-02
[SO+CO2] Iter 14 | step=0.0714 | rel_change=1.971033e-02
[SO+CO2] Iter 15 | step=0.0667 | rel_change=1.751487e-02
[SO+CO2] Iter 16 | step=0.0625 | rel_change=1.439412e-02
[SO+CO2] Iter 17 | step=0.0588 | rel_change=1.626018e-02
[SO+CO2] Iter 18 | step=0.0556 